In [112]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [113]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [96]:
Query='''

WITH
  ADC_data AS(
  SELECT
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
   CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment 
  FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' ),


Telus_customers as(

select cust_bus_cust_id,pi_cntrct_end_ts as contract_end_date,pi_cntrct_start_ts as contract_start_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-10-31' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

),



Telus_SMHM_Deact_Oct2022 AS (

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag  
from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`
--SELECT DISTINCT BAN,1 AS Telus_Churn_Flag
--FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.ad-hoc-2023-01-25` 

),


login_data AS (
  SELECT
    customer_id,
    (COUNT(DISTINCT DATE(login_dt_utc))*100/30) AS Login_consistency
  FROM
    `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
  WHERE
    DATE(login_dt_utc)< '2022-11-01'
    AND DATE(login_dt_utc)>= '2022-10-01'
  GROUP BY
    customer_id 
),


Telus_Merge AS (


select * from ADC_data as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id 

)

, Merge_data AS (
  SELECT
    ADC.*,
    CASE
      WHEN c.Telus_Churn_Flag IS NULL THEN 0
    ELSE
    c.Telus_Churn_Flag
  END
    AS Telus_Churn,
    CASE
      WHEN lg.Login_consistency IS NULL THEN "NO_User"
      WHEN lg.Login_consistency>=30 THEN 'Heavy_User'
      WHEN lg.Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Segment
  FROM
    Telus_Merge AS ADC
  LEFT JOIN
    Telus_SMHM_Deact_Oct2022 c
  ON
    --ADC.dealer_customer_id=c.CUST_ID
    ADC.BAN=c.BAN
  LEFT JOIN
    login_data lg
  ON
    lg.customer_id=ADC.customer_id )

-- SELECT
--   Package,
--   Segment,
--   COUNT(DISTINCT customer_id) AS volume,
--   SUM(Telus_Churn) AS Total_Churn,
--   ROUND(AVG(Telus_Churn)*100,2) AS Churn_rate
-- FROM
--   Merge_data
-- GROUP BY
--   Package,
--   Segment
-- ORDER BY
--   Package,
--   Segment

select * from Merge_data



'''

In [97]:
Query_updated='''



WITH
  ADC_data AS(
  SELECT
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
   CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment 
  FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' )


,Telus_customers as(

select cust_bus_cust_id,pi_cntrct_end_ts as contract_end_date,pi_cntrct_start_ts as contract_start_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-10-31' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)



,Telus_SMHM_Deact_Oct2022 AS (

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn  
from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`

)


,SHS_features as(

SELECT 
distinct ban,
AGE_BAND,
PROD_MIX_AL

 FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.base_features_scores_segments_20220731`
  
)


,Telus_Merge AS (


select ADC.*,Telus.contract_start_date,Telus.contract_end_date,Deacts.Telus_Churn,SHS.AGE_BAND,SHS.PROD_MIX_AL
from ADC_data as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join  Telus_SMHM_Deact_Oct2022 as Deacts
on ADC.BAN=Deacts.BAN
left join SHS_features as SHS
on ADC.BAN=SHS.ban

)


select * from Telus_Merge


'''

In [98]:
Camera_count_SQL='''

SELECT customer_id,combined_device_type_desc, count(distinct device_id) as count_of_camera FROM `divgpras-pr-579355.ADC_updated.ADC_Inventory_device_data` 
where date(device_install_date)<'2022-10-01'
and date(device_removal_date) is NULL
and  combined_device_type_desc='Camera'
-- or date(device_removal_date)>='2022-11-01' 
group by customer_id,combined_device_type_desc


'''



In [99]:
Camera_Count_DF=extract_bq_data(bq_client, sql=Camera_count_SQL)

In [100]:
Camera_Count_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361939 entries, 0 to 361938
Data columns (total 3 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                361939 non-null  Int64 
 1   combined_device_type_desc  361939 non-null  object
 2   count_of_camera            361939 non-null  Int64 
dtypes: Int64(2), object(1)
memory usage: 9.0+ MB


In [101]:
Camera_Count_DF.head()

,customer_id,combined_device_type_desc,count_of_camera
0,8788138,Camera,7
1,11014567,Camera,8
2,7725347,Camera,8
3,10933270,Camera,7
4,13390189,Camera,8


In [167]:
Type_of_cameras_Query= '''


SELECT distinct Customer_ID_ID as customer_id,Updated_Combined_Device_Type_DESC, count(distinct device_id) as count FROM `divgpras-pr-579355.ADC_updated.ADC_Camera_details` 
where date(Device_Install_Date_ID)<'2022-10-01'
and date(Device_Removal_Date_ID) is NULL
group by Customer_ID_ID,Updated_Combined_Device_Type_DESC



'''

In [168]:
Telus_Smart_camera_Oct='''



with data_1 as (select * from `divgpras-pr-579355.ADC_updated.Smart_Camera_Amalysis_Oct2022`)

-- , data_2 as ( SELECT ban,AGE_BAND,PROD_MIX_AL FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.base_features_scores_segments_20220731`)


, data_2 as ( select distinct ban,prod_mix
from `bi-srv-features-pr-ef5a93.ban_product.bq_ban_product_mix`
QUALIFY ROW_NUMBER() OVER (PARTITION BY ban ORDER BY part_dt DESC) = 1)



select data_1.*,data_2.prod_mix
from data_1
left join data_2 on data_1.BAN=data_2.ban


'''

In [169]:
Type_of_cameras=extract_bq_data(bq_client, sql=Type_of_cameras_Query)

In [170]:
Type_of_cameras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405449 entries, 0 to 405448
Data columns (total 3 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   customer_id                        405449 non-null  Int64 
 1   Updated_Combined_Device_Type_DESC  405449 non-null  object
 2   count                              405449 non-null  Int64 
dtypes: Int64(2), object(1)
memory usage: 10.1+ MB


In [171]:
Type_of_cameras['Updated_Combined_Device_Type_DESC'].value_counts()

Outdoor Camera                182482
Doorbell Camera - Slimline    152328
Indoor Camera                  57084
Doorbell Camera - Round        13555
Name: Updated_Combined_Device_Type_DESC, dtype: int64

In [172]:
# 'Outdoor Camera'.split(" ")[0]

In [173]:
Type_of_cameras["Device_detail_type"] = Type_of_cameras["Updated_Combined_Device_Type_DESC"].apply(lambda x: x.split(" ")[0].strip())

In [174]:
Type_of_cameras["Device_detail_type"].value_counts()

Outdoor     182482
Doorbell    165883
Indoor       57084
Name: Device_detail_type, dtype: int64

In [175]:
# pd.DataFrame(DF_monitored.groupby(['Arming_category','Login_category']).agg(
    
#     Customer_count= ('customer_id','count'),
#     Customer_count_1= ('customer_id','nunique'),
#     # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [176]:
Type_of_cameras_wide=Type_of_cameras.pivot_table(index='customer_id', columns='Device_detail_type', values='count',aggfunc = "sum").reset_index()

In [177]:
Type_of_cameras_wide.head()

Device_detail_type,customer_id,Doorbell,Indoor,Outdoor
0,299100,<NA>,<NA>,1
1,359052,1,<NA>,<NA>
2,361396,1,<NA>,<NA>
3,513943,<NA>,<NA>,1
4,528372,1,<NA>,<NA>


In [178]:
Type_of_cameras_wide.columns=['customer_id','Doorbell','indoor','Outdoor']

In [179]:
Type_of_cameras_wide.fillna(0,inplace=True)

In [180]:
Telus_Smart_camera_Oct_DF=extract_bq_data(bq_client, sql=Telus_Smart_camera_Oct)

In [181]:
Telus_Smart_camera_Oct_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326768 entries, 0 to 326767
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                326768 non-null  Int64 
 1   dealer_customer_id         326768 non-null  Int64 
 2   BAN                        326768 non-null  Int64 
 3   Package                    326768 non-null  object
 4   cust_bus_cust_id           326768 non-null  Int64 
 5   contract_end_date          326768 non-null  object
 6   contract_start_date        326768 non-null  object
 7   Telus_Churn                326768 non-null  Int64 
 8   Segment                    326768 non-null  object
 9   combined_device_type_desc  326768 non-null  object
 10  count_of_camera            326768 non-null  Int64 
 11  Camera_exact_1             326768 non-null  Int64 
 12  Camera_exact_2             326768 non-null  Int64 
 13  Camera_grt_eq_3            326768 non-null  

In [182]:
Telus_Smart_camera_Oct_DF.isna().sum()*100/Telus_Smart_camera_Oct_DF.shape[0]

customer_id                  0.000000
dealer_customer_id           0.000000
BAN                          0.000000
Package                      0.000000
cust_bus_cust_id             0.000000
contract_end_date            0.000000
contract_start_date          0.000000
Telus_Churn                  0.000000
Segment                      0.000000
combined_device_type_desc    0.000000
count_of_camera              0.000000
Camera_exact_1               0.000000
Camera_exact_2               0.000000
Camera_grt_eq_3              0.000000
prod_mix                     2.538192
dtype: float64

In [183]:
Telus_Smart_camera_Oct_DF[Telus_Smart_camera_Oct_DF['BAN']==603565171]

,customer_id,dealer_customer_id,BAN,Package,cust_bus_cust_id,contract_end_date,contract_start_date,Telus_Churn,Segment,combined_device_type_desc,count_of_camera,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3,prod_mix
12887,5551153,98316348,603565171,Monitored,98316348,2020-10-01 00:00:00,2017-10-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM


In [184]:
Telus_Smart_camera_Oct_DF_1=Telus_Smart_camera_Oct_DF.merge(Type_of_cameras_wide,on='customer_id',how='left')

In [185]:
Telus_Smart_camera_Oct_DF_1.head()

,customer_id,dealer_customer_id,BAN,Package,cust_bus_cust_id,contract_end_date,contract_start_date,Telus_Churn,Segment,combined_device_type_desc,count_of_camera,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3,prod_mix,Doorbell,indoor,Outdoor
0,6717789,98317164,603565369,Monitored,98317164,2008-11-01 00:00:00,2005-11-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
1,6584869,98316879,603564978,Monitored,98316879,2009-06-01 00:00:00,2006-06-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
2,6824131,98319221,603586645,Monitored,98319221,2009-10-01 00:00:00,2006-10-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
3,6763400,98321041,603584449,Monitored,98321041,2011-02-01 00:00:00,2008-02-01 00:00:00,0,NO_User,0,0,0,0,0,"HSIC, SING, SMHM, TTV",<NA>,<NA>,<NA>
4,6666123,98319264,603567446,Monitored,98319264,2011-05-28 00:00:00,2008-05-28 00:00:00,0,Heavy_User,0,0,0,0,0,"DIIC, SMHM",<NA>,<NA>,<NA>


In [186]:
Telus_Smart_camera_Oct_DF_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326768 entries, 0 to 326767
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                326768 non-null  Int64 
 1   dealer_customer_id         326768 non-null  Int64 
 2   BAN                        326768 non-null  Int64 
 3   Package                    326768 non-null  object
 4   cust_bus_cust_id           326768 non-null  Int64 
 5   contract_end_date          326768 non-null  object
 6   contract_start_date        326768 non-null  object
 7   Telus_Churn                326768 non-null  Int64 
 8   Segment                    326768 non-null  object
 9   combined_device_type_desc  326768 non-null  object
 10  count_of_camera            326768 non-null  Int64 
 11  Camera_exact_1             326768 non-null  Int64 
 12  Camera_exact_2             326768 non-null  Int64 
 13  Camera_grt_eq_3            326768 non-null  

In [158]:
# Telus_Smart_camera_Oct_DF_1.fillna(0,inplace=True)

In [164]:
# Telus_Smart_camera_Oct_DF_1=Telus_Smart_camera_Oct_DF_1.reset_index()

In [187]:
Telus_Smart_camera_Oct_DF_1.head()

,customer_id,dealer_customer_id,BAN,Package,cust_bus_cust_id,contract_end_date,contract_start_date,Telus_Churn,Segment,combined_device_type_desc,count_of_camera,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3,prod_mix,Doorbell,indoor,Outdoor
0,6717789,98317164,603565369,Monitored,98317164,2008-11-01 00:00:00,2005-11-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
1,6584869,98316879,603564978,Monitored,98316879,2009-06-01 00:00:00,2006-06-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
2,6824131,98319221,603586645,Monitored,98319221,2009-10-01 00:00:00,2006-10-01 00:00:00,0,NO_User,0,0,0,0,0,SMHM,<NA>,<NA>,<NA>
3,6763400,98321041,603584449,Monitored,98321041,2011-02-01 00:00:00,2008-02-01 00:00:00,0,NO_User,0,0,0,0,0,"HSIC, SING, SMHM, TTV",<NA>,<NA>,<NA>
4,6666123,98319264,603567446,Monitored,98319264,2011-05-28 00:00:00,2008-05-28 00:00:00,0,Heavy_User,0,0,0,0,0,"DIIC, SMHM",<NA>,<NA>,<NA>


In [188]:
Telus_Smart_camera_Oct_DF_1.to_csv('Telus_SmartCamera_with_camera_count_Oct2022_20230119.csv',index=False)

In [39]:
DF=extract_bq_data(bq_client, sql=Query_updated)

In [40]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340689 entries, 0 to 340688
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   customer_id           340689 non-null  Int64 
 1   dealer_customer_id    340689 non-null  object
 2   BAN                   340689 non-null  Int64 
 3   Package               340689 non-null  object
 4   Smart_camera_Segment  340689 non-null  object
 5   contract_start_date   308190 non-null  object
 6   contract_end_date     340689 non-null  object
 7   Telus_Churn           122 non-null     Int64 
 8   AGE_BAND              254275 non-null  object
 9   PROD_MIX_AL           254275 non-null  object
dtypes: Int64(3), object(7)
memory usage: 27.0+ MB


In [41]:
DF.head()

,customer_id,dealer_customer_id,BAN,Package,Smart_camera_Segment,contract_start_date,contract_end_date,Telus_Churn,AGE_BAND,PROD_MIX_AL
0,14751260,101917173,604531456,Smart_Camera,NO_User,2022-10-19 00:00:00,2025-10-19 00:00:00,<NA>,18 - 20,HS-OPTIK
1,14480551,104284927,605241388,Smart_Camera,NO_User,2022-09-05 00:00:00,2025-09-05 00:00:00,<NA>,18 - 20,SMHS
2,13928242,103781241,605042368,Smart_Camera,NO_User,2022-06-28 00:00:00,2025-06-28 00:00:00,<NA>,18 - 20,HS-OPTIK-SMHS
3,11575743,102743092,604706627,Smart_Camera,NO_User,2021-06-24 00:00:00,2024-06-24 00:00:00,<NA>,18 - 20,HS-OPTIK-SMHS
4,12807982,103705233,605012601,Smart_Camera,NO_User,2022-01-07 00:00:00,2025-01-07 00:00:00,<NA>,18 - 20,HS-SMHS


In [42]:
Merge_DF=DF.merge(Camera_Count_DF,how='left',on='customer_id')

In [43]:
Merge_DF.head()

,customer_id,dealer_customer_id,BAN,Package,Smart_camera_Segment,contract_start_date,contract_end_date,Telus_Churn,AGE_BAND,PROD_MIX_AL,combined_device_type_desc,count_of_camera
0,14751260,101917173,604531456,Smart_Camera,NO_User,2022-10-19 00:00:00,2025-10-19 00:00:00,<NA>,18 - 20,HS-OPTIK,NaN,<NA>
1,14480551,104284927,605241388,Smart_Camera,NO_User,2022-09-05 00:00:00,2025-09-05 00:00:00,<NA>,18 - 20,SMHS,Camera,3
2,13928242,103781241,605042368,Smart_Camera,NO_User,2022-06-28 00:00:00,2025-06-28 00:00:00,<NA>,18 - 20,HS-OPTIK-SMHS,NaN,<NA>
3,11575743,102743092,604706627,Smart_Camera,NO_User,2021-06-24 00:00:00,2024-06-24 00:00:00,<NA>,18 - 20,HS-OPTIK-SMHS,NaN,<NA>
4,12807982,103705233,605012601,Smart_Camera,NO_User,2022-01-07 00:00:00,2025-01-07 00:00:00,<NA>,18 - 20,HS-SMHS,NaN,<NA>


In [44]:
Merge_DF.isna().sum()*100/Merge_DF.shape[0]

customer_id                   0.000000
dealer_customer_id            0.000000
BAN                           0.000000
Package                       0.000000
Smart_camera_Segment          0.000000
contract_start_date           9.539199
contract_end_date             0.000000
Telus_Churn                  99.964190
AGE_BAND                     25.364482
PROD_MIX_AL                  25.364482
combined_device_type_desc    28.984793
count_of_camera              28.984793
dtype: float64

In [45]:
Merge_DF.fillna(0,inplace=True)

In [46]:
Merge_DF['count_of_camera'].value_counts()

1     131713
0      98748
2      71094
3      26667
4       8683
5       2298
6        854
7        348
8        184
9         50
10        25
12         9
11         7
13         5
17         3
15         1
Name: count_of_camera, dtype: Int64

In [47]:
Merge_DF['Camera_exact_1']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x==1 else 0)
Merge_DF['Camera_exact_2']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x==2 else 0)
Merge_DF['Camera_grt_eq_3']=Merge_DF['count_of_camera'].apply(lambda x: 1 if x>=3 else 0)

In [48]:
pd.DataFrame(Merge_DF.groupby(['Smart_camera_Segment','Package']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn',lambda x: x.sum()),
    Churn_rate=('Telus_Churn',lambda x: x.mean()*100)).reset_index())

,Smart_camera_Segment,Package,Customer_count,Customer_Share,Churn_total,Churn_rate
0,Heavy_User,Monitored,112837,33.806199,26,0.022575
1,Heavy_User,Smart_Automation_Plus,27122,8.061898,3,0.010923
2,Heavy_User,Smart_Camera,7480,2.236938,3,0.039365
3,Low_User,Monitored,50097,15.099108,31,0.060263
4,Low_User,Smart_Automation_Plus,28986,8.626049,11,0.037430
5,Low_User,Smart_Camera,13852,4.122528,3,0.021360
6,NO_User,Monitored,38666,12.263384,13,0.031115
7,NO_User,Smart_Automation_Plus,27177,8.275583,17,0.060297
8,NO_User,Smart_Camera,24260,7.508314,15,0.058640


In [49]:
Merge_DF.tail()

,customer_id,dealer_customer_id,BAN,Package,Smart_camera_Segment,contract_start_date,contract_end_date,Telus_Churn,AGE_BAND,PROD_MIX_AL,combined_device_type_desc,count_of_camera,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3
340684,6638020,98327807,603569947,Monitored,Low_User,2010-11-01 00:00:00,2013-11-01 00:00:00,0,Unk Age,SMHS,0,0,0,0,0
340685,8683432,98332152,603575972,Monitored,NO_User,2020-02-28 00:00:00,2023-02-28 00:00:00,0,Unk Age,SMHS,0,0,0,0,0
340686,6904863,98330064,603593384,Monitored,Heavy_User,2014-11-07 00:00:00,2017-11-07 00:00:00,0,Unk Age,SMHS,0,0,0,0,0
340687,12318896,98342619,603599419,Monitored,Heavy_User,2022-01-24 00:00:00,2025-01-24 00:00:00,0,Unk Age,SMHS,0,0,0,0,0
340688,12318896,98342619,603599419,Monitored,Heavy_User,2022-01-24 00:00:00,2025-01-24 00:00:00,0,Unk Age,SMHS,0,0,0,0,0


In [55]:
Merge_DF['Package'].value_counts()

Monitored                208395
Smart_Automation_Plus     85048
Smart_Camera              47246
Name: Package, dtype: int64

In [51]:
pd.DataFrame(Merge_DF.groupby(['Smart_camera_Segment','Package']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn',lambda x: x.sum()),
    Churn_rate=('Telus_Churn',lambda x: x.mean()*100),
    Camera_exact_1=('Camera_exact_1',lambda x: x.sum()),
    Camera_exact_2=('Camera_exact_2',lambda x: x.sum()),
    Camera_grt_eq_3=('Camera_grt_eq_3',lambda x: x.sum()),
).reset_index())

,Smart_camera_Segment,Package,Customer_count,Customer_Share,Churn_total,Churn_rate,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3
0,Heavy_User,Monitored,112837,33.806199,26,0.022575,34016,41592,27180
1,Heavy_User,Smart_Automation_Plus,27122,8.061898,3,0.010923,17989,5416,2656
2,Heavy_User,Smart_Camera,7480,2.236938,3,0.039365,4409,1910,988
3,Low_User,Monitored,50097,15.099108,31,0.060263,16613,11552,4857
4,Low_User,Smart_Automation_Plus,28986,8.626049,11,0.037430,20910,2680,837
5,Low_User,Smart_Camera,13852,4.122528,3,0.021360,10100,1909,581
6,NO_User,Monitored,38666,12.263384,13,0.031115,7275,4370,1572
7,NO_User,Smart_Automation_Plus,27177,8.275583,17,0.060297,13446,888,222
8,NO_User,Smart_Camera,24260,7.508314,15,0.058640,6955,777,241


In [56]:
Old_file_Oct=pd.read_csv('Telus_SmartCamera_with_camera_count_Oct2022_old_file.csv')

In [64]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC_updated.Smart_Camera_Amalysis_Oct2022'

bq_table_instance= bq_client.load_table_from_dataframe(Old_file_Oct, Table_BQ,job_config=config)

In [58]:
Old_file_Oct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326768 entries, 0 to 326767
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                326768 non-null  int64 
 1   dealer_customer_id         326768 non-null  int64 
 2   BAN                        326768 non-null  int64 
 3   Package                    326768 non-null  object
 4   cust_bus_cust_id           326768 non-null  int64 
 5   contract_end_date          326768 non-null  object
 6   contract_start_date        326768 non-null  object
 7   Telus_Churn                326768 non-null  int64 
 8   Segment                    326768 non-null  object
 9   combined_device_type_desc  326768 non-null  object
 10  count_of_camera            326768 non-null  int64 
 11  Camera_exact_1             326768 non-null  int64 
 12  Camera_exact_2             326768 non-null  int64 
 13  Camera_grt_eq_3            326768 non-null  

In [59]:
Old_file_Oct_1=Old_file_Oct.merge(Merge_DF[['customer_id','AGE_BAND','PROD_MIX_AL']],how='left',on='customer_id')

In [62]:
Old_file_Oct_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332225 entries, 0 to 332224
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   customer_id                332225 non-null  int64 
 1   dealer_customer_id         332225 non-null  int64 
 2   BAN                        332225 non-null  int64 
 3   Package                    332225 non-null  object
 4   cust_bus_cust_id           332225 non-null  int64 
 5   contract_end_date          332225 non-null  object
 6   contract_start_date        332225 non-null  object
 7   Telus_Churn                332225 non-null  int64 
 8   Segment                    332225 non-null  object
 9   combined_device_type_desc  332225 non-null  object
 10  count_of_camera            332225 non-null  int64 
 11  Camera_exact_1             332225 non-null  int64 
 12  Camera_exact_2             332225 non-null  int64 
 13  Camera_grt_eq_3            332225 non-null  

In [60]:
Old_file_Oct_1.head()

,customer_id,dealer_customer_id,BAN,Package,cust_bus_cust_id,contract_end_date,contract_start_date,Telus_Churn,Segment,combined_device_type_desc,count_of_camera,Camera_exact_1,Camera_exact_2,Camera_grt_eq_3,AGE_BAND,PROD_MIX_AL
0,6612761,2593136,0,Monitored,2593136,2024-11-29 00:00:00,2021-11-29 00:00:00,0,NO_User,0,0,0,0,0,0,0
1,6612761,2593136,0,Monitored,2593136,2024-11-29 00:00:00,2021-11-29 00:00:00,0,NO_User,0,0,0,0,0,66 - 70,HP-HS-OPTIK-SMHS
2,6763272,2558488,0,Monitored,2558488,2025-10-25 00:00:00,2022-10-25 00:00:00,0,NO_User,0,0,0,0,0,0,0
3,6763272,2558488,0,Monitored,2558488,2025-10-25 00:00:00,2022-10-25 00:00:00,0,NO_User,0,0,0,0,0,66 - 70,HS-OPTIK
4,7582553,1144381,0,Monitored,1144381,9999-12-31 00:00:00,0,0,NO_User,0,0,0,0,0,0,0


In [61]:
Old_file_Oct_1.to_csv('Telus_SmartCamera_with_camera_count_Oct2022.csv',index=False)

In [145]:
Sep_Data=pd.read_csv('Telus_SmartCamera_with_camera_count_Sep2022.csv')

In [146]:
Oct_Data=pd.read_csv('Telus_SmartCamera_with_camera_count_Oct2022.csv')

In [150]:
Sep_merge=Sep_Data.merge(Oct_Data[['customer_id','Segment','Telus_Churn']],how='left',on='customer_id')

In [151]:
Sep_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316932 entries, 0 to 316931
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   customer_id                316932 non-null  int64  
 1   dealer_customer_id         316932 non-null  int64  
 2   BAN                        316932 non-null  int64  
 3   Package                    316932 non-null  object 
 4   cust_bus_cust_id           316932 non-null  int64  
 5   contract_end_date          316932 non-null  object 
 6   Telus_Churn_x              316932 non-null  int64  
 7   Segment_x                  316932 non-null  object 
 8   combined_device_type_desc  316932 non-null  object 
 9   count_of_camera            316932 non-null  int64  
 10  Camera_exact_1             316932 non-null  int64  
 11  Camera_exact_2             316932 non-null  int64  
 12  Camera_grt_eq_3            316932 non-null  int64  
 13  Segment_y                  30

In [152]:
Sep_merge.to_csv('Telus_SmartCamera_with_camera_count_Sep_Oct2022.csv',index=False)

In [22]:
Customer_info=extract_bq_data(bq_client, sql=Telus_Customer_details_Sep2022)

In [23]:
Customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316932 entries, 0 to 316931
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   customer_id             316932 non-null  Int64 
 1   dealer_customer_id      316932 non-null  object
 2   dealer_name             316932 non-null  object
 3   account_type_name       316932 non-null  object
 4   customer_type_name      316932 non-null  object
 5   bacct_bus_bacct_num     316932 non-null  Int64 
 6   bacct_billg_acct_id     316932 non-null  Int64 
 7   cust_bus_cust_id        316932 non-null  object
 8   pi_prod_instnc_typ_cd   316932 non-null  object
 9   pi_prod_instnc_stat_ts  316932 non-null  object
 10  prod_instnc_ts          316932 non-null  object
 11  contract_end_date       316932 non-null  object
 12  CUST_ID                 3189 non-null    object
 13  Telus_Churn_Flag        3189 non-null    Int64 
dtypes: Int64(4), object(10)
memory usage

In [24]:
Customer_info['account_type_name'].value_counts()

Security System             195342
Awareness and Automation     76741
Standalone                   44849
Name: account_type_name, dtype: int64

In [53]:
Customer_info['account_type_name'].value_counts(normalize=True)*100

Security System             61.635303
Awareness and Automation    24.213711
Standalone                  14.150985
Name: account_type_name, dtype: float64

In [28]:
Customer_info['Telus_Churn_Flag'].fillna(0,inplace=True)

In [60]:
pd.DataFrame(Customer_info.groupby(['account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Customer_info.shape[0]),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,account_type_name,Customer_count,Customer_Share,Churn_total,Churn_rate
0,Awareness and Automation,76741,24.213711,825,1.075045
1,Security System,195342,61.635303,1490,0.762765
2,Standalone,44849,14.150985,874,1.948761


In [29]:
Login_SQL='''

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-10-01' and date(login_dt_utc)>= '2022-09-01' 

)



select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql ) as count_of_dates_logins
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-10-01' and date(login_dt_utc)>= '2022-09-01'
group by customer_id
order by number_of_login_days desc


'''

In [30]:
Login_DF=extract_bq_data(bq_client, sql=Login_SQL)

In [31]:
max_number_login_dates=Login_DF['count_of_dates_logins'].max()
Login_DF['Login_Consistency']=Login_DF['number_of_login_days'].apply(lambda x: x*100/max_number_login_dates)

In [32]:
Login_DF.head()

,customer_id,sum_login_count,number_of_login_days,count_of_dates_logins,Login_Consistency
0,4633153,102,30,30,100.0
1,8479829,153,30,30,100.0
2,12055682,147,30,30,100.0
3,11249211,189,30,30,100.0
4,9557034,165,30,30,100.0


In [35]:
Login_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416342 entries, 0 to 416341
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   customer_id            416342 non-null  Int64  
 1   sum_login_count        416342 non-null  Int64  
 2   number_of_login_days   416342 non-null  Int64  
 3   count_of_dates_logins  416342 non-null  Int64  
 4   Login_Consistency      416342 non-null  float64
dtypes: Int64(4), float64(1)
memory usage: 17.5 MB


In [40]:
pd.DataFrame(Login_DF['Login_Consistency'].value_counts(normalize=True)*100).reset_index().sort_values(by='index',ascending=False)

,index,Login_Consistency
0,100.000000,27.352993
4,96.666667,3.937148
6,93.333333,2.992011
9,90.000000,2.458796
11,86.666667,2.227496
14,83.333333,2.061526
16,80.000000,1.973618
19,76.666667,1.880425
18,73.333333,1.891714
22,70.000000,1.767777


In [67]:
Merge_DF=Customer_info.merge(Login_DF,on='customer_id',how='left')

In [68]:
Merge_DF.isna().sum()*100/Merge_DF.shape[0]

customer_id                0.000000
dealer_customer_id         0.000000
dealer_name                0.000000
account_type_name          0.000000
customer_type_name         0.000000
bacct_bus_bacct_num        0.000000
bacct_billg_acct_id        0.000000
cust_bus_cust_id           0.000000
pi_prod_instnc_typ_cd      0.000000
pi_prod_instnc_stat_ts     0.000000
prod_instnc_ts             0.000000
contract_end_date          0.000000
CUST_ID                   98.993790
Telus_Churn_Flag           0.000000
sum_login_count           25.619691
number_of_login_days      25.619691
count_of_dates_logins     25.619691
Login_Consistency         25.619691
dtype: float64

In [65]:
Merge_DF.fillna(0,inplace=True)

In [38]:
Merge_DF.head()

,customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name,bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,contract_end_date,CUST_ID,Telus_Churn_Flag,sum_login_count,number_of_login_days,count_of_dates_logins,Login_Consistency
0,8359929,100221804,TELUS Communications Inc.,Security System,Customer,604141100,100652002,100221804,SMHM,2019-12-24,2022-09-30 00:00:00+00:00,2022-12-23 00:00:00,0,0,60,25,30,83.333333
1,11762596,100301814,TELUS Communications Inc.,Security System,Customer,604148517,100757545,100301814,SMHM,2020-01-20,2022-09-30 00:00:00+00:00,2023-01-21 00:00:00,0,0,0,0,0,0.000000
2,13167647,100401951,TELUS Communications Inc.,Awareness and Automation,Commitment,604179396,100951568,100401951,SMHM,2022-03-02,2022-09-30 00:00:00+00:00,2025-03-02 00:00:00,0,0,0,0,0,0.000000
3,11058833,100470120,TELUS Communications Inc.,Awareness and Automation,Customer,604201986,101071952,100470120,SMHM,2021-04-20,2022-09-30 00:00:00+00:00,2024-04-20 00:00:00,0,0,172,30,30,100.000000
4,12966345,10062970,TELUS Communications Inc.,Security System,Customer,225553339,2815375,10062970,SMHM,2022-01-28,2022-09-30 00:00:00+00:00,2025-01-28 00:00:00,0,0,278,30,30,100.000000


In [47]:
def Segment_making(row,login_consistency_threshold=50):
    
    if row['Login_Consistency']>=login_consistency_threshold :
        return 'Heavy_Usage'
    elif row['Login_Consistency']==0 :
        return 'No_Usage'
    elif row['Login_Consistency']< login_consistency_threshold :
        return 'Low_Usage'
    else:
        return "Unassigned"

In [48]:
Merge_DF['Segment']=Merge_DF.apply(Segment_making,axis=1)

In [49]:
Merge_DF['Segment'].value_counts(normalize=True)*100

Heavy_Usage    45.173413
Low_Usage      29.206896
No_Usage       25.619691
Name: Segment, dtype: float64

In [50]:
def Telus_plans(row):
    if row['account_type_name'] == 'Standalone':
        return 'Smart_Camera'
    elif row['account_type_name'] =='Awareness and Automation':
        return 'Smart_Automation_Plus' 
    else:
        return 'Monitored'

Merge_DF['Telus_plans'] = Merge_DF.apply(Telus_plans, axis=1)

In [59]:
Merge_DF['Telus_plans'].value_counts()

Monitored                195342
Smart_Automation_Plus     76741
Smart_Camera              44849
Name: Telus_plans, dtype: int64

In [51]:
Merge_DF['Telus_plans'].value_counts(normalize=True)*100

Monitored                61.635303
Smart_Automation_Plus    24.213711
Smart_Camera             14.150985
Name: Telus_plans, dtype: float64

In [57]:
pd.DataFrame(Merge_DF.groupby(['Telus_plans','Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF.shape[0]),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Telus_plans,Segment,Customer_count,Customer_Share,Churn_total,Churn_rate
0,Monitored,Heavy_Usage,110226,34.779069,25,0.022681
1,Monitored,Low_Usage,49375,15.579052,31,0.062785
2,Monitored,No_Usage,35741,11.277182,1434,4.012199
3,Smart_Automation_Plus,Heavy_Usage,25399,8.014022,2,0.007874
4,Smart_Automation_Plus,Low_Usage,28604,9.025280,4,0.013984
5,Smart_Automation_Plus,No_Usage,22738,7.174410,819,3.601900
6,Smart_Camera,Heavy_Usage,7544,2.380321,2,0.026511
7,Smart_Camera,Low_Usage,14587,4.602565,0,0.000000
8,Smart_Camera,No_Usage,22718,7.168099,872,3.838366


In [58]:
Merge_DF.to_csv('Telus_SmartCamera_Sep2022.csv',index=False)